<a href="https://colab.research.google.com/github/Shugyla1234/Yessenov-Data-Lab/blob/main/Disasters_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio openai

In [ ]:
import openai
openai.key

In [ ]:
import io
import PIL
import base64
import gradio as gr
from openai import OpenAI

In [ ]:
# I hid the API key due to privacy and security
from openai import AzureOpenAI

OPENAI_API_KEY = "YOUR REAL KEY"
OPENAI_ENDPOINT
OPENAI_DEPLOYMENT
OPENAI_API_VERSION
OPENAI_DEPLOYMENT_EMBEDDING

client = AzureOpenAI(
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
)

In [ ]:

import gradio as gr

def survival_assistant(location, threat, family, no_net):
    prompt = f"""
Civilian Survival Request

Crisis Info:
- Location: {location}
- Threat: {threat}
- Family: {family}
- Internet/electricity: {"No" if no_net else "Yes"}

Give calm, short instructions:
- What to do
- What to pack
- How to stay safe
"""
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content


disaster_types = [
    "Airstrike",
    "Evacuation",
    "Occupation",
    "Blackout",
    "Earthquake",
    "Tsunami",
    "Nuclear Explosion",
    "Flood",
    "Wildfire",
    "Chemical Leak",
    "Missile Strike",
    "Civil Unrest"
]

gr.Interface(
    fn=survival_assistant,
    inputs=[
        gr.Textbox(label="📍 Location", placeholder="e.g. Gaza"),
        gr.Dropdown(disaster_types, label="🧨 Threat"),
        gr.Textbox(label="👨‍👩‍👧 Family", placeholder="e.g. 2 kids, grandma"),
        gr.Checkbox(label="🚫 No Internet")
    ],
    outputs=gr.Textbox(label="🛡️ Survival Plan", lines=12),
    title="🛡️ AI Civil Survival Assistant",
    description="Get a survival plan tailored to your situation — including natural and man-made disasters."
).launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b585b9b20365a87cba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#Offline Disaster Network & Locator App (Works with coordinates)

In [ ]:

import math
OPENAI_DEPLOYMENT
shelters = [
    {"name": "Central Metro Bunker", "lat": 40.7128, "lon": -74.0060, "type": "shelter"},
    {"name": "Red Cross Tent", "lat": 40.7132, "lon": -74.0005, "type": "medical"},
    {"name": "UN Food Station", "lat": 40.7140, "lon": -74.0021, "type": "food"},
]
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    d_lat, d_lon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    a = math.sin(d_lat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2)**2
    return R * (2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)))
def find_nearest(lat, lon, need):
    filtered = [s for s in shelters if need.lower() in s["type"]]
    sorted_places = sorted(filtered, key=lambda x: haversine(lat, lon, x["lat"], x["lon"]))
    return sorted_places[0] if sorted_places else None
def disaster_locator(lat, lon, need):
    nearest = find_nearest(lat, lon, need)
    if not nearest:
        return "No resources nearby. Try a different need.", None

    shelter_info = f"{nearest['name']} at ({nearest['lat']:.4f}, {nearest['lon']:.4f})"

    gpt_prompt = f"""
You're an emergency assistant helping a disaster victim.

They're located at ({lat}, {lon}), and they need: {need}.

You found this location: {shelter_info}.

Give calm, clear directions to reach this place and what to expect there.
"""

    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": gpt_prompt}]
    )

    return response.choices[0].message.content, shelter_info


gr.Interface(
    fn=disaster_locator,
    inputs=[
        gr.Number(label="📍 Latitude"),
        gr.Number(label="📍 Longitude"),
        gr.Dropdown(["shelter", "medical", "food"], label="What do you need?")
    ],
    outputs=[
        gr.Textbox(label="📢 Instructions"),
        gr.Textbox(label="📌 Nearest Location")
    ],
    title="🆘 Disaster Survival Locator"
).launch(debug=True)


In [ ]:
#Offline Disaster Network & Locator App (Version 2)

In [ ]:
# I hid the API key due to privacy and security
from openai import AzureOpenAI

OPENAI_API_KEY = "YOUR REAL KEY"
OPENAI_ENDPOINT
OPENAI_DEPLOYMENT
OPENAI_API_VERSION
OPENAI_DEPLOYMENT_EMBEDDING

client = AzureOpenAI(
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
)

shelters = [
    {"name": "AlmaU Underground Shelter", "location": "Almaty, near AlmaU", "type": "shelter"},
    {"name": "KazRed Emergency Tent", "location": "Almaty, Dostyk Avenue", "type": "medical"},
    {"name": "UN Food Station", "location": "Almaty, Sayran Bus Terminal", "type": "food"},
    {"name": "NYC Central Metro Bunker", "location": "New York", "type": "shelter"},
]

def disaster_locator(location_text, need):
    shelter_list = [s for s in shelters if need.lower() in s["type"]]
    if not shelter_list:
        return "No resources found for this need.", ""
    gpt_prompt = f"""
You're a calm and clear emergency assistant helping a disaster victim.

They are currently at: {location_text}
They are looking for: {need}

These are possible support centers:

{chr(10).join([f"- {s['name']} at {s['location']}" for s in shelter_list])}

Based on their location, give a brief explanation of which shelter is most likely closest,
and how to reach it. Offer simple survival guidance.
"""
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": gpt_prompt}],
        temperature=0.5,
        max_tokens=600
    )
    instructions = response.choices[0].message.content.strip()
    top_shelter = shelter_list[0]["name"] + " at " + shelter_list[0]["location"]

    return instructions, top_shelter
gr.Interface(
    fn=disaster_locator,
    inputs=[
        gr.Textbox(label="📍 Where are you right now?", placeholder="e.g. near Al-Quds Hospital, Gaza"),
        gr.Dropdown(["shelter", "medical", "food"], label="🚨 What do you need?")
    ],
    outputs=[
        gr.Textbox(label="📢 Emergency Instructions", lines=10),
        gr.Textbox(label="📌 Nearest Known Shelter")
    ],
    title="🆘 AI Disaster Assistant (Location-Aware)",
    description="Type your location and need. The assistant will suggest where to go and how."
).launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9af4c29d9d20134a42.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9af4c29d9d20134a42.gradio.live


In [ ]:
#AI Lost Contact Assistant
## Helping people reconnect during disasters or war when traditional communication fails

In [ ]:
# @title
def lost_contact_assistant(last_seen, destination, contact_method, emotional_msg):
    prompt = f"""
You are an emergency contact assistant. Someone is trying to reconnect with a missing person during a disaster.

Here is the situation:
- Last seen location: {last_seen}
- They were heading towards: {destination}
- Communication available: {contact_method}
- Add emotional message to comfort the user: {emotional_msg}

Your tasks:
- Suggest where the person might be now, based on geography and crisis logic
- Recommend how to try to contact them (e.g., radio, relay stations, messages)
- Give clear, calm guidance
- End with a short emotional comfort message
"""

    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

gr.Interface(
    fn=lost_contact_assistant,
    inputs=[
        gr.Textbox(label="📍 Last Seen Location", placeholder="e.g. North Gaza, water tower"),
        gr.Textbox(label="➡️ Direction They Were Going", placeholder="e.g. South toward Khan Younis"),
        gr.Radio(["Internet", "Radio", "None"], label="📡 Communication Available"),
        gr.Textbox(label="💬 Emotional Message", placeholder="e.g. I hope you're safe, please come back..."),
    ],
    outputs=gr.Textbox(label="📢 Contact Plan and Support Message", lines=12),
    title="🛰️ AI Lost Contact Assistant",
    description="Helps reconnect with people lost during war, evacuation, or disasters using GPT survival logic."
).launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://90a14f16b882a41332.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://90a14f16b882a41332.gradio.live


In [ ]:
# Voice Input + Text-to-Speech
## Let the user speak the emergency situation instead of typing it+ Read the instructions aloud for kids or elderly who can't read

In [ ]:
!pip install pyttsx3

In [ ]:
!apt install -y espeak

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install SpeechRecognition

In [ ]:
import gradio as gr
from openai import AzureOpenAI
import tempfile
import os
import subprocess
import speech_recognition as sr
import pyttsx3

# I hid the API key due to privacy and security
from openai import AzureOpenAI

OPENAI_API_KEY = "YOUR REAL KEY"
OPENAI_ENDPOINT
OPENAI_DEPLOYMENT
OPENAI_API_VERSION
OPENAI_DEPLOYMENT_EMBEDDING

client = AzureOpenAI(
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
)


def convert_to_wav(input_path):
    if input_path is None or not os.path.exists(input_path):
        raise ValueError("⚠️ Файл не загружен или не существует.")
    output_path = input_path + ".wav"
    subprocess.run(["ffmpeg", "-i", input_path, output_path, "-y", "-loglevel", "quiet"])
    if not os.path.exists(output_path):
        raise ValueError("⚠️ Не удалось сконвертировать аудио.")
    return output_path


def voice_input_survival(audio_path):
    print(f"Получен audio_path: {audio_path}")
    wav_path = convert_to_wav(audio_path)


    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
        try:
            scenario = recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "🤷 Не удалось распознать речь.", "", None


    prompt = f"""
You are a survival expert. Write a short, calm 3–5 step guide for this situation:
Scenario: {scenario}
Steps should be clear and simple.
"""
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=400
    )
    instructions = response.choices[0].message.content.strip()


    engine = pyttsx3.init()
    temp_audio_path = tempfile.mktemp(suffix=".mp3")
    engine.save_to_file(instructions, temp_audio_path)
    engine.runAndWait()

    return scenario, instructions, temp_audio_path

gr.Interface(
    fn=voice_input_survival,
    inputs=gr.Audio(label="🎤 Speak Your Disaster Scenario", type="filepath"),
    outputs=[
        gr.Textbox(label="📝 Recognized Scenario"),
        gr.Textbox(label="📋 Survival Instructions", lines=10),
        gr.Audio(label="🔊 Audio Guide", type="filepath")
    ],
    title="🗣️ Voice-Based Disaster Survival Guide",
    description="Speak your emergency situation and get clear voice instructions."
).launch(debug=True)



def voice_input_survival(audio_path):
    print(f"Получен audio_path: {audio_path}")
    wav_path = convert_to_wav(audio_path)

    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
        try:
            scenario = recognizer.recognize_google(audio)
            print(f"🎙️ Распознано: {scenario}")
        except sr.UnknownValueError:
            return "🤷 Не удалось распознать речь.", "", None

    prompt = f"""
You are a survival expert. Write a short, calm 3–5 step guide for this situation:
Scenario: {scenario}
Steps should be clear and simple.
"""
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=400
    )
    instructions = response.choices[0].message.content.strip()
    print(f"📋 Инструкции: {instructions}")


    engine = pyttsx3.init()
    temp_wav_path = tempfile.mktemp(suffix=".wav")
    engine.save_to_file(instructions, temp_wav_path)
    engine.runAndWait()
    print(f"✅ Audio saved to WAV: {temp_wav_path}")


    temp_mp3_path = temp_wav_path.replace(".wav", ".mp3")
    subprocess.run(["ffmpeg", "-i", temp_wav_path, temp_mp3_path, "-y", "-loglevel", "quiet"])
    print(f"✅ Converted to MP3: {temp_mp3_path}")

    return scenario, instructions, temp_mp3_path


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bc4cea8efc78acc902.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Получен audio_path: /tmp/gradio/a6368edeac8346ff593dd24eb553d24dc6307855bdc121cc628fd3b788fb4ac2/ttsMP3.com_VoiceText_2025-7-4_13-12-29.mp3
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://bc4cea8efc78acc902.gradio.live


In [ ]:
# @title
import gradio as gr
from openai import AzureOpenAI
import tempfile
import os
import subprocess
import speech_recognition as sr
import pyttsx3


OPENAI_API_KEY = "YOUR REAL KEY"
OPENAI_ENDPOINT
OPENAI_DEPLOYMENT
OPENAI_API_VERSION
OPENAI_DEPLOYMENT_EMBEDDING

client = AzureOpenAI(
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
)


def convert_to_wav(input_path):
    output_path = input_path + ".wav"
    subprocess.run(["ffmpeg", "-i", input_path, output_path, "-y", "-loglevel", "quiet"])
    return output_path

def voice_input_survival(audio_path):
    wav_path = convert_to_wav(audio_path)

    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
        scenario = recognizer.recognize_google(audio)

    prompt = f"""
You are a survival expert. Write a short, calm 3–5 step guide for this situation:
Scenario: {scenario}
Steps should be clear and simple.
"""
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=400
    )
    instructions = response.choices[0].message.content

    temp_audio = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
    engine = pyttsx3.init()
    engine.save_to_file(instructions, temp_audio.name)
    engine.runAndWait()

    return scenario, instructions, temp_audio.name

gr.Interface(
    fn=voice_input_survival,
    inputs=gr.Audio(sources="microphone", label="🎤 Speak Your Disaster Scenario", streaming=True),
    outputs=[
        gr.Textbox(label="📝 Recognized Scenario"),
        gr.Textbox(label="📋 Survival Instructions", lines=10),
        gr.Audio(label="🔊 Audio Guide")
    ],
    title="🗣️ Voice-Based Disaster Survival Guide",
    description="Speak your emergency situation and get clear voice instructions."
).launch(debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:766: UserWarning: Streaming components are only supported in live interfaces.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a62ffeb133e701e66f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://a62ffeb133e701e66f.gradio.live
